## PROJET DATA PART1 ALS ___ EL MEKKAOUI FAYSSAL __ ELAMMARI SOUHAIL

### Importer les packages 

In [59]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.sql.types.IntegerType



import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.sql.types.IntegerType


### Creation de la Class Rating

In [60]:
case class Rating(userId: Int, movieId: Int, rating: Float, timestamp: Long)
def parseRating(str: String): Rating = {
  val fields = str.split("::")
  assert(fields.size == 4)
  Rating(fields(0).toInt, fields(1).toInt, fields(2).toFloat, fields(3).toLong)
}


defined class Rating
parseRating: (str: String)Rating


### Lecture du fichier csv et transformation des types string en Int

In [79]:
 
val df = spark.read.option("header",true)
    .csv("C:/Users/user/Desktop/ratings.csv")
  
val ratin = df.withColumn("userId",col("userId").cast(IntegerType))
val rating = ratin.withColumn("movieId",col("movieId").cast(IntegerType))
val ratings = rating.withColumn("rating",col("rating").cast(IntegerType))






df: org.apache.spark.sql.DataFrame = [userId: string, movieId: string ... 2 more fields]
ratin: org.apache.spark.sql.DataFrame = [userId: int, movieId: string ... 2 more fields]
rating: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]
ratings: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


### Data training

In [80]:
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))



training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: int, movieId: int ... 2 more fields]


### Construction du model de recommendation

In [81]:

val als = new ALS()
  .setMaxIter(5)
  .setRegParam(0.01)
  .setImplicitPrefs(true)
  .setUserCol("userId")
  .setItemCol("movieId")
  .setRatingCol("rating")
  


als: org.apache.spark.ml.recommendation.ALS = als_cc361a249ee6


In [82]:
val model = als.fit(training)

model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_cc361a249ee6, rank=10


In [83]:
model.setColdStartStrategy("drop")
val predictions = model.transform(test)

val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating")
  .setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

// Generate top 10 movie recommendations for each user
val userRecs = model.recommendForAllUsers(10)
// Generate top 10 user recommendations for each movie
val movieRecs = model.recommendForAllItems(10)

userRecs.show(4)

Root-mean-square error = 3.1103882701306165


predictions: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_6cef6d5a1100, metricName=rmse, throughOrigin=false
rmse: Double = 3.1103882701306165
userRecs: org.apache.spark.sql.DataFrame = [userId: int, recommendations: array<struct<movieId:int,rating:float>>]
movieRecs: org.apache.spark.sql.DataFrame = [movieId: int, recommendations: array<struct<userId:int,rating:float>>]


### UserRecs : reccommandation des films pour chaque utilisateur
### movieRecs : recommandation des Users pour chaque film

In [86]:
userRecs.show(10)
movieRecs.show(10)


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[2571, 0.5197625...|
|   463|[[318, 0.40459764...|
|   496|[[2571, 0.3016072...|
|   148|[[68954, 0.559856...|
|   540|[[2571, 0.5525483...|
|   392|[[2571, 0.3379473...|
|   243|[[380, 0.77037174...|
|    31|[[356, 0.6742091]...|
|   516|[[2571, 0.3240862...|
|   580|[[2571, 1.314706]...|
+------+--------------------+
only showing top 10 rows

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[45, 1.2400022],...|
|   4900|[[414, 0.62973076...|
|   6620|[[474, 1.0908772]...|
|   7340|[[414, 0.697927],...|
|  32460|[[105, 0.6989926]...|
|  54190|[[89, 0.81857014]...|
|    471|[[474, 1.1514992]...|
|   1591|[[274, 0.97374576...|
| 140541|[[89, 0.42497468]...|
|   1342|[[387, 0.64905417...|
+-------+--------------------+
only showing top 10 rows

